<a href="https://colab.research.google.com/github/athiniotim/Final_Essay_Forest_Fires/blob/main/forest_fires_kathimerini_teasers_scraper_Michaela_Athinioti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

In [ ]:
def get_webpage_soup(url):

  headers = {
        "User-Agent": "Mozilla/5.0"
    }

  response = requests.get(url, headers=headers)


  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:

    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = []
  for article in articlesL:
    # title
    try:
      title = article.find("a", {"class": "py-4 mainlink"}).text.strip()
    except:
      try:
        title = article.find("p", class_=lambda c: c and "title is-5 medium_title" in c).text.strip()
      except:
        title=""
    # article url
    try:
      article_url = article.find("div", {"class": "content"}).a["href"]
    except:
      try:
        article_url = article.find("a", {"class": "py-4 mainlink"})["href"]
      except:
        article_url=""
    # date & time
    try:
      datetime_str = article.find("time")["datetime"]
      date_str = datetime_str.split("T")[0]
      time_str = datetime_str.split("T")[1].split("+")[0]
    except:
      try:
        datetime_str = article.find("div", {"class": "card-date"}).text.strip()
        dt_partsL = datetime_str.split(".")
        date_str = f"{dt_partsL[2]}-{dt_partsL[1]}-{dt_partsL[0]}"
        time_str = ""
      except:
        date_str = ""
        time_str = ""
    # section
    try:
      section = article.find("p", class_=lambda c: c and "subtitle" in c).text.strip()
    except:
      section = ""
    # excerpt
    try:
      excerpt = article.find("div", class_= lambda c: c and "lead" in c).text.strip()
    except:
      excerpt = ""
    # author
    try:
      author = article.find("span", {"class":"mb-2 author-link"}).text.strip()
    except:
      author = ""

    articleD = {"title": title, "article_url": article_url, "date": date_str, "time": time_str, "section": section, "excerpt": excerpt, "author": author}

    articles_dataL.append(articleD)

  return articles_dataL

In [ ]:
url = "https://www.kathimerini.gr/search/%CF%84%CF%81%CE%BF%CF%87%CE%B1%CE%AF%CE%BF/page/1/?cat=%CE%9A%CE%BF%CE%B9%CE%BD%CF%89%CE%BD%CE%AF%CE%B1&usr&dt_from=01-01-2022&dt_to=31-12-2022"
print(len(url.split("/")))
url.split("/")

8


['https:',
 '',
 'www.kathimerini.gr',
 'search',
 '%CF%84%CF%81%CE%BF%CF%87%CE%B1%CE%AF%CE%BF',
 'page',
 '1',
 '?cat=%CE%9A%CE%BF%CE%B9%CE%BD%CF%89%CE%BD%CE%AF%CE%B1&usr&dt_from=01-01-2022&dt_to=31-12-2022']

In [ ]:
def scrape_kathimerini_teaser_pages(scrape_url, start_page_nbr=1, end_page_nbr=1):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1):
    if "search" in scrape_url:
      search_term = scrape_url.split("/")[4]
      if len(scrape_url.split("/")) > 7:
        search_params = scrape_url.split("/")[-1]
      else:
        search_params = ""
      url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
    else:
      category = scrape_url.split("/")[3]
      url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"

    print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")


    soup = get_webpage_soup(url)

    articlesL = soup.find_all("div", {"class":"nx-article"})


    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)


    all_articles_dataL.extend(page_articles_dataL)


    delay = random.uniform(1, 3)
    time.sleep(delay)

  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

Παραθέτω εναλλακτικό κώδικα που είχα χρησιμοποιήσει επειδή σε κάποια φάση δεν μου έκανε scrape όλες τις σελίδες που δοκίμαζα, λόγω error.

In [ ]:
# def scrape_kathimerini_teaser_pages(scrape_url, start_page_nbr=1, end_page_nbr=1):

#   all_articles_dataL = []

#   print(f"\rScraping started", end="")

#   for page_nbr in range(start_page_nbr, end_page_nbr+1):

#     if "search" in scrape_url:
#       search_term = scrape_url.split("/")[4]
#       if len(scrape_url.split("/")) > 7:

#         parts = scrape_url.split('/')

#         page_index = parts.index('page')

#         search_params_parts = parts[page_index + 2:]
#         search_params = '/'.join(search_params_parts)
#       else:
#         search_params = ""


#       if search_params:
#           url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/{search_params}'
#       else:
#           url = f'https://www.kathimerini.gr/search/{search_term}/page/{page_nbr}/'

#     else:
#       category = scrape_url.split("/")[3]
#       url = f"https://www.kathimerini.gr/{category}/page/{page_nbr}/"

#     print(f"\rScraping page {page_nbr}: {url}" + " "*2, end="")


#     soup = get_webpage_soup(url)


#     if soup is None:
#         print(f"\nSkipping page {page_nbr} due to scraping error.")
#         continue


#     articlesL = soup.find_all("div", {"class":"nx-article"})


#     page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

#    n
#     all_articles_dataL.extend(page_articles_dataL)


#     delay = random.uniform(1, 3)
#     time.sleep(delay)


#   df = pd.DataFrame(all_articles_dataL)
#   print(f"\rScraping completed. Fetshed {len(df)} articles.")

#   return df

In [ ]:
scrape_url = "https://www.kathimerini.gr/search/%CE%B4%CE%B1%CF%83%CE%B9%CE%BA%CE%AD%CF%82%20%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AD%CF%82/page/2/?cat&usr&dt_from=01-01-2023&dt_to=31-12-2024"
kathimerini_fires_teasers_df = scrape_kathimerini_teaser_pages(scrape_url, 1, 44)
kathimerini_fires_teasers_df

Scraping page 1: https://www.kathimerini.gr/search/%CE%B4%CE%B1%CF%83%CE%B9%CE%BA%CE%AD%CF%82%20%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AD%CF%82/page/1/?cat&usr&dt_from=01-01-2023&dt_to=31-12-2024  https://www.kathimerini.gr/search/%CE%B4%CE%B1%CF%83%CE%B9%CE%BA%CE%AD%CF%82%20%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AD%CF%82/page/1/?cat&usr&dt_from=01-01-2023&dt_to=31-12-2024 was scraped successfully
Scraping page 2: https://www.kathimerini.gr/search/%CE%B4%CE%B1%CF%83%CE%B9%CE%BA%CE%AD%CF%82%20%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AD%CF%82/page/2/?cat&usr&dt_from=01-01-2023&dt_to=31-12-2024  https://www.kathimerini.gr/search/%CE%B4%CE%B1%CF%83%CE%B9%CE%BA%CE%AD%CF%82%20%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AD%CF%82/page/2/?cat&usr&dt_from=01-01-2023&dt_to=31-12-2024 was scraped successfully
Scraping page 3: https://www.kathimerini.gr/search/%CE%B4%CE%B1%CF%83%CE%B9%CE%BA%CE%AD%CF%82%20%CF%80%CF%85%CF%81%CE%BA%CE%B1%CE%B3%CE%B9%CE%AD%CF%82/page/3/?cat&usr&dt_

,title,article_url,date,time,section,excerpt,author
0,"ΟΗΕ: Το 2024, η κλιματική αλλαγή προκάλεσε ακρ...",https://www.kathimerini.gr/world/563396869/oie...,2024-12-30,,ΚΟΣΜΟΣ,Σύμφωνα με τον Παγκόσμιο Μετεωρολογικό Οργανισ...,
1,"«Ζούμε σε μια νέα, επικίνδυνη εποχή» – Η υπερθ...",https://www.kathimerini.gr/life/environment/56...,2024-12-27,,ΠΕΡΙΒΑΛΛΟΝ,"Το 2024, η πιο ζεστή χρονιά που έχει καταγραφε...",
2,Ερευνα: Οι 14 μεγάλες πυρκαγιές του 2024,https://www.kathimerini.gr/life/environment/56...,2024-12-17,,ΠΕΡΙΒΑΛΛΟΝ,Στην έκθεση του Αστεροσκοπείου και της WWF Ελλ...,
3,Πορτογαλία και Ισπανία έζησαν τον πιο ζεστό Νο...,https://www.kathimerini.gr/world/563358376/por...,2024-12-05,,ΚΟΣΜΟΣ,Η μέση θερμοκρασία του Νοεμβρίου στην Πορτογαλ...,
4,Δύο νέες πλατφόρμες για την ανάλυση του κλίματος,https://www.kathimerini.gr/society/563356729/d...,2024-12-05,,ΚΟΙΝΩΝΙΑ,Δύο νέους ψηφιακούς κόμβους ανάλυσης της ατμόσ...,
...,...,...,...,...,...,...,...
643,Καλιφόρνια: Τουλάχιστον 14 νεκροί από τη σφοδρ...,https://www.kathimerini.gr/world/562221718/kal...,2023-01-10,,ΚΟΣΜΟΣ,«Τα θύματα είναι περισσότερα από εκείνα που σκ...,
644,Copernicus: Το 2022 ήταν το δεύτερο πιο ζεστό ...,https://www.kathimerini.gr/world/562221523/cop...,2023-01-10,,ΚΟΣΜΟΣ,«Ζούμε ήδη τις καταστροφικές συνέπειες του θερ...,
645,Βραζιλία: Οι ρίζες της εξέγερσης των μπολσοναρ...,https://www.kathimerini.gr/world/562221112/vra...,2023-01-10,,ΚΟΣΜΟΣ,Τα σχέδια του προέδρου Λούλα για την προστασία...,
646,"Καλιφόρνια – Κακοκαιρία: Ένα νεκρό παιδί, χιλι...",https://www.kathimerini.gr/world/562215640/kal...,2023-01-05,,ΚΟΣΜΟΣ,"Ο «ατμοσφαιρικός ποταμός», όπως ονομάζεται το ...",


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
kathimerini_fires_teasers_df.to_csv("/content/drive/MyDrive/Final Essay ΕΔΔΕ.2 Αθηνιώτη Μιχαέλα/clean csv/forest_fires_kathimerini_teaser_csv.csv", index=False)